In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [ ]:
def ex_tag(sid, page):
    ### 뉴스 분야(sid)와 페이지(page)를 입력하면 그에 대한 링크들을 리스트로 추출하는 함수 ###

    ## 1.
    url = f"https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={sid}"\
    "#&date=%2000:00:00&page={page}"
    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    a_tag = soup.find_all("a")

    ## 2.
    tag_lst = []
    for a in a_tag:
        if "href" in a.attrs:  # href가 있는것만 고르는 것
            if (f"sid={sid}" in a["href"]) and ("article" in a["href"]):
                tag_lst.append(a["href"])

    return tag_lst

In [ ]:
def re_tag(sid):
    ### 특정 분야의 100페이지까지의 뉴스의 링크를 수집하여 중복 제거한 리스트로 변환하는 함수 ###
    re_lst = []
    for i in tqdm(range(100)):
        lst = ex_tag(sid, i+1)
        re_lst.extend(lst)

    # 중복 제거
    re_set = set(re_lst)
    re_lst = list(re_set)

    return re_lst

In [ ]:
lst = re_tag(100)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
lst

['https://n.news.naver.com/mnews/article/002/0002306317?sid=100',
 'https://n.news.naver.com/mnews/article/422/0000627196?sid=100',
 'https://n.news.naver.com/mnews/article/119/0002765086?sid=100',
 'https://n.news.naver.com/mnews/article/023/0003796889?sid=100',
 'https://n.news.naver.com/mnews/article/656/0000068003?sid=100',
 'https://n.news.naver.com/mnews/article/003/0012184785?sid=100',
 'https://n.news.naver.com/mnews/article/214/0001309261?sid=100',
 'https://n.news.naver.com/mnews/article/009/0005208379?sid=100',
 'https://n.news.naver.com/mnews/article/023/0003796860?sid=100',
 'https://n.news.naver.com/mnews/article/030/0003151792?sid=100',
 'https://n.news.naver.com/mnews/article/421/0007150038?sid=100',
 'https://n.news.naver.com/mnews/article/214/0001309201?sid=100',
 'https://n.news.naver.com/mnews/article/003/0012182099?sid=100',
 'https://n.news.naver.com/mnews/article/057/0001778097?sid=100',
 'https://n.news.naver.com/mnews/article/660/0000048292?sid=100',
 'https://

In [ ]:
def re_tag(sid):
    ### 특정 분야의 100페이지까지의 뉴스의 링크를 수집하여 중복 제거한 리스트로 변환하는 함수 ###
    re_lst = []
    for i in tqdm(range(100)):
        lst = ex_tag(sid, i+1)
        re_lst.extend(lst)

    # 중복 제거
    re_set = set(re_lst)
    re_lst = list(re_set)

    return re_lst

In [ ]:
all_hrefs = {}
sids = [i for i in range(100,106)]  # 분야 리스트

# 각 분야별로 링크 수집해서 딕셔너리에 저장
for sid in sids:
    sid_data = re_tag(sid)
    all_hrefs[sid] = sid_data

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
all_href = {100: lst}

In [ ]:
def art_crawl(all_hrefs, sid, index):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수

    Args:
        all_hrefs(dict): 각 분야별로 100페이지까지 링크를 수집한 딕셔너리 (key: 분야(sid), value: 링크)
        sid(int): 분야 [100: 정치, 101: 경제, 102: 사회, 103: 생활/문화, 104: 세계, 105: IT/과학]
        index(int): 링크의 인덱스

    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리

    """
    art_dic = {}

    ## 1.
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"

    url = all_hrefs[sid][index]
    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")

    ## 2.
    # 제목 수집
    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)

    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)

    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)

    ## 3.
    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str

    return art_dic

In [ ]:
# 모든 섹션의 데이터 수집 (제목, 날짜, 본문, section, url)
section_lst = [s for s in range(100, 106)]
artdic_lst = []

for i in tqdm(range(len(all_href[100]))):
    art_dic = art_crawl(all_href, 100, i)
    art_dic["section"] = 100
    art_dic["url"] = all_href[100][i]
    artdic_lst.append(art_dic)

  0%|          | 0/486 [00:00<?, ?it/s]

In [ ]:
artdic_lst

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(artdic_lst)

In [ ]:
df['main'][0]

'혁신위 \'당 화합\' 제안 수용했지만…총선 핵심 보직은 \'윤핵관\' 돌려막기국민의힘이 홍준표 대구시장, 김재원 최고위원, 이준석 전 대표 등에 대한 징계 취소를 의결했다. 인요한 혁신위가 \'당 화합\'을 명분으로 꺼낸 1호 제안을 수용한 것이지만, 총선 핵심 보직 중 하나인 인재영입위원장에는 직전 사무총장을 지낸 \'윤핵관\' 이철규 의원을 임명해 통합 의지의 진정성에 물음표가 찍힌다. 박정하 국민의힘 수석대변인은 2일 최고위원회의 뒤 기자들과 만나 "최고위원회의는 혁신위의 당 통합을 위한 화합 제안을 대승적 차원에서 수용하기로 했다"며 "오늘 징계 처분 취소가 의결된 대상자는 이 전 대표, 홍 시장, 김재원 전 최고위원, 김철근 강서병 당협위원장"이라고 밝혔다. 김기현 국민의힘 대표는 앞서 회의 모두발언에서 "혁신위는 1호 안건으로 당내 화합을 제시했다"며 "과거 윤리위의 징계 결정이 나름 합리적인 기준을 통해 이뤄져 존중돼야 하지만 혁신위 제안도 존중돼야 마땅하다. 다른 의견이 있을 수 있지만 혁신위의 진정성을 수용하는 것이 옳다는 의견을 갖고 있다"고 했었다. 박 대변인은 또 "김재원 전 최고위원의 자진사퇴로 인한 궐위와 관련해 보궐선거 선관위를 구성했다"며 "보궐선거 시점은 대강 11월 말이 될 것"이라고 밝혔다. \'5.18 전문 헌법 수록 반대\', \'전광훈 목사 우파 천하 통일\' 등 발언으로 징계를 받았던 김 전 최고위원은 전날 최고위원직을 내려놨는데 이를 두고는 징계 취소 전 당 지도부의 정치적 부담을 덜어주기 위한 것으로 풀이됐다. 박 대변인은 인요한 혁신위가 전날 회의에서 \'동일 지역구 3선 이상 연임 금지\'를 검토한 데 대해서는 "그 건에 대해서는 (최고위에서) 논의되지 않았다"며 "인 위원장 말씀이 아직 우리에게 전달 안 돼 이후에 말씀드리겠다"고 했다. 한편 박 대변인은 이날 "인재영입위원장으로는 이철규 전 사무총장을 의결했다"며 "전직 사무총장으로 인재 영입 활동을 오래 전부터 해 와 업무 연속성을 감안했다"고 밝혔다. 